In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn

# Create a random input tensor
input_tensor = torch.randn(1, 1000, 6, 7, 4)  # (batch_size, in_channels, depth, height, width)

# Instantiate the nn.Conv3d layer
conv_layer = nn.Conv3d(in_channels=1000, out_channels=1000, kernel_size=(1,1,6), stride=2, padding=1)

# Pass the input tensor through the conv_layer
output_tensor = conv_layer(input_tensor)

# Print the shapes of the input and output tensors
print("Input shape:", input_tensor.shape)
print("Output shape:", output_tensor.shape)


Input shape: torch.Size([1, 1000, 6, 7, 4])
Output shape: torch.Size([1, 1000, 4, 5, 1])


In [ ]:
input_tensor = torch.randn(1, 1000, 6, 7, 4)  # (batch_size, in_channels, depth, height, width)

# Instantiate the nn.Conv3d layer
conv_layer = nn.Conv3d(in_channels=1000, out_channels=1000, kernel_size=(6,1,4), stride=1, padding=0)
# Pass the input tensor through the conv_layer
output_tensor = conv_layer(input_tensor)

# Print the shapes of the input and output tensors
print("Input shape:", input_tensor.shape)
print("Output shape:", output_tensor.shape)

Input shape: torch.Size([1, 1000, 6, 7, 4])
Output shape: torch.Size([1, 1000, 1, 7, 1])


In [ ]:
input_tensor = torch.randn(1, 1000, 1, 7, 1)  # (batch_size, in_channels, depth, height, width)

# Instantiate the nn.Conv3d layer
conv_layer = nn.ConvTranspose3d(in_channels=1000, out_channels=1000, kernel_size=(6, 1, 4), stride=1, padding=0)
# Pass the input tensor through the conv_layer
output_tensor = conv_layer(input_tensor)

# Print the shapes of the input and output tensors
print("Input shape:", input_tensor.shape)
print("Output shape:", output_tensor.shape)

Input shape: torch.Size([1, 1000, 1, 7, 1])
Output shape: torch.Size([1, 1000, 6, 7, 4])


In [ ]:
import torch
import torch.nn as nn

class CustomActivation(nn.Module):
    def __init__(self, scaling_factor=8):
        super(CustomActivation, self).__init__()
        self.scaling_factor = scaling_factor

    def forward(self, x):
        # Apply your custom activation function
        return self.scaling_factor * torch.tanh(x)

# Define the encoder architecture
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1000, out_channels=1500, kernel_size=(1, 1, 4), stride=1, padding=0)
        self.conv2 = nn.Conv3d(in_channels=1500, out_channels=1000, kernel_size=(6, 1, 1), stride=1, padding=0)


    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))

        return x

# Define the decoder architecture
class Decoder(nn.Module):
    def __init__(self, scaling_factor=8):
        super(Decoder, self).__init__()
        self.conv1 = nn.ConvTranspose3d(in_channels=1000, out_channels=1500, kernel_size=(6, 1, 1), stride=1, padding=0)
        self.conv2 = nn.ConvTranspose3d(in_channels=1500, out_channels=1000, kernel_size=(1, 1, 4), stride=1, padding=0)
        self.activation = CustomActivation(scaling_factor)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.activation(x)
        return x

# Combine the encoder and decoder into a single autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, scaling_factor=8):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder(scaling_factor)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded


In [ ]:
import torch.optim as optim
import numpy as np
fileindex = [2,3,8]
for i in range(1,10):
    # Prepare training data
    train_data = scipy.io.loadmat(f'/content/drive/MyDrive/4DGraz/4D_{i:02d}T_P.mat')['data']
    valid_data = scipy.io.loadmat(f'/content/drive/MyDrive/4DGraz/4D_{i:02d}E_P.mat')['data']


    max = np.max(train_data)
    min = np.min(train_data)
    model = None
    if max > min*(-1):
        model = Autoencoder(scaling_factor=max).to('cuda')
    else:
        model = Autoencoder(scaling_factor=min).to('cuda')
    # Define the autoencoder model


    # Define the loss function
    criterion = nn.MSELoss()

    # Define the optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

    # Set the number of training epochs
    num_epochs = 5

    # Initialize variables for tracking the best model
    best_loss = float('inf')
    best_model_path = 'best_model.pth'

    # Training loop
    for epoch in range(num_epochs):
        running_loss = 0.0

        # Iterate over the training data
        for inputs in train_data:
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            inputs = torch.from_numpy(inputs).float()
            inputs = inputs.unsqueeze(0).permute(0, 4, 1, 2, 3).to('cuda')
            encoded, decoded = model(inputs)

            # Calculate the loss
            loss = criterion(decoded, inputs)

            # Backward pass
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Track the running loss
            running_loss += loss.item()

        # Compute the average loss for the epoch
        epoch_loss = running_loss / len(train_data)

        # Print the epoch loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

        # Validate the model
        valid_loss = 0.0
        for inputs in valid_data:
            with torch.no_grad():
                inputs = torch.from_numpy(inputs).float()
                inputs = inputs.unsqueeze(0).permute(0, 4, 1, 2, 3).to('cuda')
                encoded, decoded = model(inputs)
                loss = criterion(decoded, inputs)
                valid_loss += loss.item()
        valid_loss /= len(valid_data)

        # Check if the current model is the best
        if valid_loss < best_loss:
            best_loss = valid_loss
            # Save the model checkpoint
            torch.save(model.state_dict(), best_model_path)

    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()

    train = []
    for inputs in train_data:
        inputs = torch.from_numpy(inputs).float()
        inputs = inputs.unsqueeze(0).permute(0, 4, 1, 2, 3).to('cuda')
        encoded, _ = model(inputs)
        train.append(encoded.view(7, 1000).cpu().detach().numpy())

    valid = []
    for inputs in valid_data:
        inputs = torch.from_numpy(inputs).float()
        inputs = inputs.unsqueeze(0).permute(0, 4, 1, 2, 3).to('cuda')
        encoded, _ = model(inputs)
        valid.append(encoded.view(7, 1000).cpu().detach().numpy())

    train_data = {'data': np.array(train), 'label': scipy.io.loadmat(f'/content/drive/MyDrive/4DGraz/4D_{i:02d}T_P.mat')['label']}
    scipy.io.savemat(f'compressed4D_{i:02d}T_P.mat', train_data)

    valid_data = {'data': np.array(valid), 'label': scipy.io.loadmat(f'/content/drive/MyDrive/4DGraz/4D_{i:02d}E_P.mat')['label']}
    scipy.io.savemat(f'compressed4D_{i:02d}E_P.mat', valid_data)


Epoch [1/5], Loss: 0.1313
Epoch [2/5], Loss: 0.1313
Epoch [3/5], Loss: 0.1312
Epoch [4/5], Loss: 0.1312
Epoch [5/5], Loss: 0.1312
Epoch [1/5], Loss: 6.8753
Epoch [2/5], Loss: 0.1390
Epoch [3/5], Loss: 0.1314
Epoch [4/5], Loss: 0.1314
Epoch [5/5], Loss: 0.1316
Epoch [1/5], Loss: 2.2302
Epoch [2/5], Loss: 0.1432
Epoch [3/5], Loss: 0.1313
Epoch [4/5], Loss: 0.1313
Epoch [5/5], Loss: 0.1313
Epoch [1/5], Loss: 0.3300
Epoch [2/5], Loss: 0.1320
Epoch [3/5], Loss: 0.1318
Epoch [4/5], Loss: 0.1318
Epoch [5/5], Loss: 0.1317
Epoch [1/5], Loss: 0.1312
Epoch [2/5], Loss: 0.1311
Epoch [3/5], Loss: 0.1311
Epoch [4/5], Loss: 0.1311
Epoch [5/5], Loss: 38.7918
Epoch [1/5], Loss: 0.1314
Epoch [2/5], Loss: 37.4166
Epoch [3/5], Loss: 103.8283
Epoch [4/5], Loss: 102.5299
Epoch [5/5], Loss: 75.5680
Epoch [1/5], Loss: 0.1312
Epoch [2/5], Loss: 0.1311
Epoch [3/5], Loss: 0.1311
Epoch [4/5], Loss: 0.1311
Epoch [5/5], Loss: 0.1311
Epoch [1/5], Loss: 0.1816
Epoch [2/5], Loss: 5.9282
Epoch [3/5], Loss: 0.1435
Epoch

In [ ]:
import glob
import shutil

# Get a list of .mat files matching the pattern
file_list = glob.glob('*.mat')

# Specify the destination directory in Google Drive
destination_directory = '/content/drive/MyDrive/4DCompressedGraz/'

# Iterate over each .mat file
for file_path in file_list:
    # Copy the file to the destination directory
    shutil.copy(file_path, destination_directory)